<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks




In [ ]:
#OpenPose
import os
from os.path import exists, join, basename, splitext

#Change the drive to my mounted gdrive
%cd /content

git_repo_url = 'https://github.com/thofes/openpose'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake becaue of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

In [ ]:
# Yolo muss nur der Pfad angegebn werden es werden alle Fotos darin gemacht
#detect.py und plot.py muss bearbeitet werden
%cd /content/yolov5
path = '/content.gdrive/MyDrive/Fotos/1/2.jpg'
path_2 = '/content.gdrive/MyDrive/Fotos/1'
!python detect.py --weights yolov5s.pt --img 640 --max-det 1 --conf 0.25 --save-crop --project '{path_2}' --source '{path}'

In [ ]:
#OpenPose

%cd /content

folder_path = '/content.gdrive/MyDrive/Fotos/1/person'
!cd openpose && ./build/examples/openpose/openpose.bin  --number_people_max -1 --keypoint_scale 3 --disable_blending --image_dir '{folder_path}' --display 0 --write_images '{folder_path}' --write_json '{folder_path}' 
#--model_pose 'MPI'
#--write_images '{folder_path}'

In [ ]:
PATH = "/content.gdrive/My Drive/Train"
Hills= []
try:
    Hills = next(os.walk(PATH))[1]
except StopIteration:
    pass
print(Hills)

for h in Hills:
  Path_Hills = PATH + '/' + h
  print(Path_Hills)
  Bibs = []
  try:
    Bibs = next(os.walk(Path_Hills))[1]
  except StopIteration:
    pass
  print(Bibs)

  for b in Bibs:
    Path_Bib = Path_Hills + '/' + b
    print(Path_Bib)
    Pics = []
    try:
      Pics = next(os.walk(Path_Bib))[2]
    except StopIteration:
      pass
    print(Pics)

    for p in Pics:
      Path_Pic = Path_Bib + '/' + p
      %cd /content/yolov5
      !python detect.py --weights yolov5s.pt --img 640 --conf 0.5 --save-crop --project '{Path_Bib}' --source '{Path_Pic}'
#--max-det 1 
    %cd /content
    folder_path = Path_Bib + '/' + 'person'
    !cd openpose && ./build/examples/openpose/openpose.bin  --disable_blending --number_people_max -1 --keypoint_scale 3 --image_dir '{folder_path}' --display 0  --write_images '{folder_path}' --write_json '{folder_path}' 





In [ ]:
 # Test zu erstellen eines Datensets mit Json

import json
import os

dataset_path = "/content.gdrive/My Drive/Dataset" 
json_path = "/content.gdrive/My Drive//Dataset/data_10.json"

# dictionary to store mapping, labels, and Keypoints
data = {
    "Decision": [],
    "labels":[],
    "keypoints":[]
}


for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

# ensure we're processing a genre sub-folder level
  if dirpath is not dataset_path:
# save genre label (i.e., sub-folder name) in the mapping
    semantic_label = dirpath.split("/")[-1]
    data["Decision"].append(semantic_label)
    print("\nProcessing: {}".format(semantic_label))

    print("dirPath: ", dirpath, "sematicpath: ", semantic_label, "filenames: ", filenames )

    %cd /content/yolov5
    !python detect.py --weights yolov5s.pt --img 640 --conf 0.5 --save-crop --project '{dirpath}' --source '{dirpath}'

# process all audio files in genre sub-dir
    for f in filenames:
      data["keypoints"].append(i-1)
      data["labels"].append(i-1)


    #%cd /content
    #folder_path = Path_save + '/exp/crops/person'
    #!cd openpose && ./build/examples/openpose/openpose.bin  --disable_blending --number_people_max -1 --keypoint_scale 3 --image_dir '{folder_path}' --display 0  --write_images '{folder_path}' --write_json '{folder_path}'
    

print(data)

with open(json_path, "w") as fp:
  json.dump(data, fp, indent=4)
